In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
pd.set_option('display.max_columns', 30)

In [10]:
pwd!

'/Users/suneethi/Documents/nssdatasc/dq4_nashville_metro/dq4-metro-beetles'

In [15]:
metro=pd.read_csv('data/hubNashville__311__Service_Requests.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
metro.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,Preferred Language,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,108110,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/10/18 16:14,3/5/19 21:34,Phone,NaN,NaN,NaN,False,NaN,False,413 Freda Villa,MADISON,8.0,37115,36.243960,-86.727210,"(36.24396, -86.72721)"
1,122078,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/7/18 14:50,12/7/18 14:50,Phone,NaN,NaN,NaN,False,NaN,True,"4906 Dakota Ave, Nashville, TN 37209, United S...",NASHVILLE,24.0,37209,36.147680,-86.846440,"(36.14768, -86.84644)"
2,113730,Closed,"Streets, Roads & Sidewalks",Request New Sign,NaN,11/1/18 10:00,11/6/18 11:15,Phone,NaN,NaN,NaN,False,NaN,True,"873 Granada Ave, Nashville, TN 37206, United S...",NASHVILLE,5.0,37206,36.189230,-86.756430,"(36.18923, -86.75643)"
3,124624,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/19/18 17:47,12/19/18 17:47,Phone,NaN,NaN,NaN,False,NaN,False,4304 Lindawood Dr,NASHVILLE,34.0,37215,36.097154,-86.831878,"(36.0971543, -86.8318783)"
4,108241,Closed,"Streets, Roads & Sidewalks",Street Lighting,Report Outage,10/10/18 14:49,10/24/18 16:27,hubNashville Community,NaN,NaN,NaN,False,NaN,True,"336 Blackman Rd, Nashville, TN 37211, United S...",NASHVILLE,26.0,37211,36.074580,-86.746960,"(36.07458, -86.74696)"


In [5]:
metro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250199 entries, 0 to 250198
Data columns (total 21 columns):
Request #                     250199 non-null int64
Status                        250199 non-null object
Request Type                  250131 non-null object
Subrequest Type               215888 non-null object
Additional Subrequest Type    141961 non-null object
Date / Time Opened            250199 non-null object
Date / Time Closed            243714 non-null object
Request Origin                250199 non-null object
System of Origin ID           22601 non-null object
Contact Type                  25483 non-null object
Preferred Language            0 non-null float64
State Issue                   250199 non-null bool
Parent Request                2713 non-null float64
Closed When Created           250199 non-null bool
Address                       218256 non-null object
City                          216613 non-null object
Council District              216500 non-null float64

In [6]:
metro=metro.drop(columns=['Request Origin', 'System of Origin ID', 'Contact Type', 
                              'Preferred Language', 'State Issue'])

In [7]:
metro.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,108110,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/10/18 16:14,3/5/19 21:34,NaN,False,413 Freda Villa,MADISON,8.0,37115,36.243960,-86.727210,"(36.24396, -86.72721)"
1,122078,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/7/18 14:50,12/7/18 14:50,NaN,True,"4906 Dakota Ave, Nashville, TN 37209, United S...",NASHVILLE,24.0,37209,36.147680,-86.846440,"(36.14768, -86.84644)"
2,113730,Closed,"Streets, Roads & Sidewalks",Request New Sign,NaN,11/1/18 10:00,11/6/18 11:15,NaN,True,"873 Granada Ave, Nashville, TN 37206, United S...",NASHVILLE,5.0,37206,36.189230,-86.756430,"(36.18923, -86.75643)"
3,124624,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/19/18 17:47,12/19/18 17:47,NaN,False,4304 Lindawood Dr,NASHVILLE,34.0,37215,36.097154,-86.831878,"(36.0971543, -86.8318783)"
4,108241,Closed,"Streets, Roads & Sidewalks",Street Lighting,Report Outage,10/10/18 14:49,10/24/18 16:27,NaN,True,"336 Blackman Rd, Nashville, TN 37211, United S...",NASHVILLE,26.0,37211,36.074580,-86.746960,"(36.07458, -86.74696)"


In [8]:
metro=metro.rename(columns={'Request #': 'Request_id', 'Request Type':'Request_type',
                                'Subrequest Type': 'Subrequest_type', 
                                'Additional Subrequest Type': 'Additional_subrequest',
                               'Date / Time Opened' :'dt_opened', 'Date / Time Closed':'dt_closed',
                               'Council District': 'Council_dist', 'ZIP':'Zip', 'Latitude':'Lat', 
                               'Longitude':'Long', 'Mapped Location': 'Location'})

In [9]:
metro.head(30)

,Request_id,Status,Request_type,Subrequest_type,Additional_subrequest,dt_opened,dt_closed,Parent Request,Closed When Created,Address,City,Council_dist,Zip,Lat,Long,Location
0,108110,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/10/18 16:14,3/5/19 21:34,NaN,False,413 Freda Villa,MADISON,8.0,37115,36.243960,-86.727210,"(36.24396, -86.72721)"
1,122078,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/7/18 14:50,12/7/18 14:50,NaN,True,"4906 Dakota Ave, Nashville, TN 37209, United S...",NASHVILLE,24.0,37209,36.147680,-86.846440,"(36.14768, -86.84644)"
2,113730,Closed,"Streets, Roads & Sidewalks",Request New Sign,NaN,11/1/18 10:00,11/6/18 11:15,NaN,True,"873 Granada Ave, Nashville, TN 37206, United S...",NASHVILLE,5.0,37206,36.189230,-86.756430,"(36.18923, -86.75643)"
3,124624,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/19/18 17:47,12/19/18 17:47,NaN,False,4304 Lindawood Dr,NASHVILLE,34.0,37215,36.097154,-86.831878,"(36.0971543, -86.8318783)"
4,108241,Closed,"Streets, Roads & Sidewalks",Street Lighting,Report Outage,10/10/18 14:49,10/24/18 16:27,NaN,True,"336 Blackman Rd, Nashville, TN 37211, United S...",NASHVILLE,26.0,37211,36.074580,-86.746960,"(36.07458, -86.74696)"
5,100318,Closed,"Streets, Roads & Sidewalks",Request New Sign,NaN,9/18/18 13:10,11/7/18 11:04,NaN,True,"Buchanan St & Delta Ave, Nashville, TN 37208, USA",NASHVILLE,21.0,37208,36.180530,-86.801600,"(36.18053, -86.8016)"
6,96553,Closed,"Trash, Recycling & Litter",New Service/Activate Service,New Construction Residential Recycle,9/10/18 14:12,10/5/18 4:05,NaN,True,"119 S Graycroft Ave, Nashville, TN 37115, Unit...",MADISON,8.0,37115,36.260000,-86.732410,"(36.26, -86.73241)"
7,105253,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,10/2/18 12:18,10/5/18 11:15,NaN,True,"2614 Old Buena Vista Rd, Nashville, TN 37218, ...",NASHVILLE,2.0,37218,36.207600,-86.811810,"(36.2076, -86.81181)"
8,101980,Closed,Property Violations,Residential Complaint,NaN,9/21/18 19:15,9/24/18 9:48,NaN,True,"1805 Eastside Ave, Nashville, TN 37206, United...",NASHVILLE,6.0,37206,36.168240,-86.738720,"(36.16824, -86.73872)"
9,102951,Closed,"Trash, Recycling & Litter",Missed Pickup - Recycle Cart Service,Recycle - Curbside/Alley Missed Pickup,9/25/18 14:44,9/27/18 4:11,NaN,True,1825 underwood st,NASHVILLE,21.0,37208,36.172890,-86.810160,"(36.17289, -86.81016)"


In [23]:
metro_misd=metro[(metro['Subrequest_type']=='Missed Pickup - Recycle Cart Service') | (metro['Subrequest_type']==
                                                                                'Missed Pickup - Trash Cart Service')
         | (metro['Subrequest_type']=='Missed Pickup - Cart Service') | (metro['Subrequest_type']=='Missed Pickup - Dumpster Service')]